In [ ]:
!pip install streamlit
!pip install streamlit_option_menu
!pip install EasyOCR
!pip install mysql
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.5-cp310-cp310-l

In [ ]:
%%writefile neth.py
import pandas as pd
import numpy as np
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
import sqlite3 as sql
#import mysql.connector as sql
from PIL import Image
import os
import re
from pyngrok import ngrok

# SETTING PAGE CONFIGURATIONS
# icon = Image.open("/content/streamlit.jpg")
st.set_page_config(page_title="BizCardX: Extracting Business Card Data with OCR",
                  # page_icon=icon,
                   layout="wide",
                   initial_sidebar_state="expanded",
                   menu_items={'About': """# This OCR app is created by *NATARAJAN T*!"""})


# SETTING-UP BACKGROUND IMAGE
def setting_background():
    st.markdown(f"""
    <style>
        .stApp {{
            background: linear-gradient(to right,  #87CEEB, #87CEEB);
            background-size: cover;
            transition: background 0.5s ease;
        }}
        h1,h2,h3,h4,h5,h6 {{
            color: #f3f3f3;
            font-family: 'Open Sans', Lato;
        }}
        .stButton>button {{
            color: #4e4376;
            background-color: #ececec ;
            transition: all 0.2s ease-in-out;
        }}
        .stButton>button:hover {{
            color: #f3f3f3;
            background-color: #FCFCFB;
        }}
        .stTextInput>div>div>input {{
            color: #4e4376;
            background-color: #f3f3f3;
        }}
    </style>
    """,unsafe_allow_html=True)
setting_background()

#setting Project Heading
st.markdown('<p style="text-align:center; color:yellow; font-size:40px; font-weight:bold;">BizCardX: Extracting Business Card Data with OCR</p>', unsafe_allow_html=True)
st.divider()


# CREATING OPTION MENU
selected = option_menu(None, ["Home", "Upload & Extract", "Modify or Delete"],
                       icons=["house", "cloud-upload", "pencil-square"],
                       default_index=0,
                       orientation="horizontal",
                       styles={"nav-link": {"font-size": "35px", "text-align": "centre", "margin": "-2px",
                                           "--hover-color": "#6495ED"},
                               "icon": {"font-size": "35px"},
                               "container": {"max-width": "6000px"},
                               "nav-link-selected": {"background-color": "#6495ED"}})

# Connect to SQLite database (or create a new one if it doesn't exist)
conn = sql.connect('project_biscard.db')

# Create a cursor object
cursor = conn.cursor()

# Business_card: Create a table
cursor.execute("CREATE TABLE IF NOT EXISTS Extracted_data (Company_Name TEXT, Card_Holder TEXT, Designation TEXT, Mobile_Number TEXT UNIQUE, Email TEXT, Website TEXT, Area TEXT, City TEXT, State TEXT, Pin_Code TEXT)")



# INITIALIZING THE EasyOCR READER
reader = easyocr.Reader(['en'],gpu=False)

# HOME MENU
if selected == "Home":
    col1 , col2 = st.columns(2)
    with col1:
        # st.image(Image.open("/content/extract.jpg"),width=500)
        st.markdown("## :green[**Technologies Used**:] Python,easy OCR, Streamlit, SQLite, Pandas")
    with col2:
       st.write("## :green[**About :**] Bizcard is a Python application designed to extract information from business cards.")
       st.write('## The main purpose of Bizcard is to automate the process of extracting key details from business card images, such as the name, designation, company, contact information, and other relevant data. By leveraging the power of OCR (Optical Character Recognition) provided by EasyOCR, Bizcard is able to extract text from the images.')

# UPLOAD AND EXTRACT MENU

if selected == "Upload & Extract":

  if st.button(":blue[Database]"):

      # Execute SQL query to fetch data from the Extracted_data table
      cursor.execute("SELECT Company_Name, Card_Holder, Designation, Mobile_Number, Email, Website, Area, City, State, Pin_Code FROM Extracted_data")

      # Fetch all the data
      data = cursor.fetchall()

      # Convert the fetched data into a pandas DataFrame
      df = pd.DataFrame(data, columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number", "Email", "Website", "Area", "City", "State", "Pin_Code"])


      # Add a new column for serial numbers
      df['Serial_Number'] = range(1, len(df) + 1)

      # Set the serial number as the index
      df.set_index('Serial_Number', inplace=True)

      st.write(df)

  st.subheader(":blue[Upload a Business Card]")
  uploaded_card = st.file_uploader("upload here", label_visibility="collapsed", type=["png", "jpeg", "jpg"])

  if uploaded_card is not None:
      def save_card(uploaded_card):
          uploaded_cards_dir = os.path.join(os.getcwd(), "uploaded_cards")
          # Create the directory if it doesn't exist
          os.makedirs(uploaded_cards_dir, exist_ok=True)
          with open(os.path.join(uploaded_cards_dir, uploaded_card.name), "wb") as f:
              f.write(uploaded_card.getbuffer())

      save_card(uploaded_card)

      # DISPLAYING THE UPLOADED CARD
      col1, col2 = st.columns(2, gap="large")
      with col1:

          st.markdown("### Uploaded Business Card")
          st.image(uploaded_card)
      # DISPLAYING THE Datas in Card
      with col2:

          with st.spinner("Please wait processing image..."):

            uploaded_file = uploaded_card.read()

          reader = easyocr.Reader(['en'],gpu=False)
          results=reader.readtext((uploaded_file))

          results_text=[]




          for text in results:

            results_text.append(text[1])

          #st.write(results_text)

          def get_data(results_text):
            data = {
                "company_name": [],
                "card_holder": [],
                "designation": [],
                "mobile_number": [],
                "email": [],
                "website": [],
                "area": [],
                "city": [],
                "state": [],
                "pin_code": []
            }

            for ind, i in enumerate(results_text):
                # To get WEBSITE_URL
                if isinstance(i, str) and ("www " in i.lower() or "www." in i.lower()):
                    data["website"].append(i)


                # To get EMAIL ID
                elif "@" in i:
                    data["email"].append(i)

                # To get MOBILE NUMBER
                elif "-" in i:
                  data["mobile_number"].append(i)
                  if len(data["mobile_number"]) ==2:
                      data["mobile_number"].append(i)


                # To get CARD HOLDER NAME
                elif ind == 0:  # Assuming card holder name is at index 0
                    data["card_holder"].append(i)

                # To get DESIGNATION
                elif ind == 1:
                    data["designation"].append(i)

                # To get AREA
                if isinstance(i, str) and (re.findall('^[0-9].+, [a-zA-Z]+', i) or re.findall('[0-9] [a-zA-Z]+', i)):
                    data["area"].append(i.split(',')[0])

                # To get CITY NAME
                match1 = re.findall('.+St , ([a-zA-Z]+).+', i)
                match2 = re.findall('.+St,, ([a-zA-Z]+).+', i)
                match3 = re.findall('^[E].*', i)
                if match1:
                    data["city"].append(match1[0])
                elif match2:
                    data["city"].append(match2[0])
                elif match3:
                    data["city"].append(match3[0])

                # To get STATE
                state_match = re.findall('[a-zA-Z]{9} +[0-9]', i)
                if state_match:
                    data["state"].append(i[:9])
                elif re.findall('^[0-9].+, ([a-zA-Z]+);', i):
                    data["state"].append(i.split()[-1])
                if len(data["state"]) == 2:
                    data["state"].pop(0)

                # To get PINCODE
                if isinstance(i, str) and len(i) >= 6 and i.isdigit():
                    data["pin_code"].append(i)
                elif isinstance(i, str) and re.findall('[a-zA-Z]{9} +[0-9]', i):
                    data["pin_code"].append(i[10:])

                # To get COMPANY NAME
                elif ind == len(results_text)-1:
                  data["company_name"].append(i)

            return data

          result_data=get_data(results_text)
          #st.write(result_data)

          #FUNCTION TO CREATE DATAFRAME
          def create_df(data):
              df = pd.DataFrame([data])
              return df
          df_1 = create_df(result_data)
          st.success("### :green Data Extracted!")
          st.write(df_1)


          if st.button('Upload to Database'):
            for i, row in df_1.iterrows():

              insert_query = "INSERT INTO Extracted_data (company_name, card_holder, designation, mobile_number, email, website, area, city, state, pin_code) \
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"


              values = (
                    row["company_name"][0],
                    row["card_holder"][0],
                    row["designation"][0],
                    row["mobile_number"][0],
                    row["email"][0],
                    row["website"][0],
                    row["area"][0],
                    row["city"][0],
                    row["state"][0],
                    row["pin_code"][0]
                )

              try:

                  cursor.execute(insert_query, values)
                  conn.commit()
                  st.success("### Data Uploaded to Database!")

              except Exception as e:
                  st.write(f"Error occurred: {e}")



# Modify Menu
if selected == "Modify or Delete":
  col1,col2=st.columns(2)

  with col1:

    st.markdown("### Update Details in Database ###")


    cursor.execute("SELECT card_holder,mobile_number FROM Extracted_data")
    result = cursor.fetchall()
    business_cards = {}
    for row in result:
        business_cards[row[0]] = row[0]
    selected_card = st.selectbox("Select a card holder name to update", list(business_cards.keys()))

    st.markdown("#### Modify Data")
    cursor.execute("select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from Extracted_data WHERE card_holder=?",
                    (selected_card,))
    result = cursor.fetchone()

    # DISPLAYING ALL THE INFORMATIONS
    company_name = st.text_input("Company_Name", result[0])
    card_holder = st.text_input("Card_Holder", result[1])
    designation = st.text_input("Designation", result[2])
    mobile_number = st.text_input("Mobile_Number", result[3])
    email = st.text_input("Email", result[4])
    website = st.text_input("Website", result[5])
    area = st.text_input("Area", result[6])
    city = st.text_input("City", result[7])
    state = st.text_input("State", result[8])
    pin_code = st.text_input("Pin_Code", result[9])

    if st.button("Update in Database"):

      # Update the information for the selected business card in the database
      cursor.execute("""UPDATE Extracted_data SET company_name=?,card_holder=?,designation=?,mobile_number=?,email=?,website=?,area=?,city=?,state=?,pin_code=?
                                    WHERE card_holder=?""", (company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,selected_card))
      conn.commit()
      st.success("Data updated in database successfully.")

  with col2:

    st.markdown("### Delete Record in Databse ###")

    cursor.execute("SELECT card_holder FROM Extracted_data")
    result = cursor.fetchall()
    business_cards = {}
    for row in result:
        business_cards[row[0]] = row[0]
    selected_card = st.selectbox("Select a card holder name to Delete", list(business_cards.keys()))
    st.write(f"### You have selected :red[**{selected_card}'s**] card to delete")
    st.write("#### Proceed to delete this card?")

    if st.button("Yes"):
        cursor.execute(f"DELETE FROM Extracted_data WHERE card_holder='{selected_card}'")
        conn.commit()
        st.success("Data deleted from database.")

    if st.button("Updated Database"):

      cursor.execute("SELECT * FROM Extracted_data")
      updated_data=cursor.fetchall()

      # Convert the fetched data into a pandas DataFrame
      df_updated = pd.DataFrame(updated_data, columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number", "Email", "Website", "Area", "City", "State", "Pin_Code"])


      # Add a new column for serial numbers
      df_updated['Serial_Number'] = range(1, len(df_updated) + 1)

      # Set the serial number as the index
      df_updated.set_index('Serial_Number', inplace=True)

      st.write(df_updated)


Writing neth.py


In [ ]:

# Restart Streamlit app
!streamlit run neth.py &>/dev/null&

In [ ]:
!ngrok config add-authtoken 2d5PSPuXVXoo3vCfE3aOFinu7zq_2ApqbF5Y4yqLHBe2VYfmg



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(addr='8501')
print(public_url)

NgrokTunnel: "https://1888-34-171-91-185.ngrok-free.app" -> "http://localhost:8501"
